In [9]:
# Import the necessary libraries
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
import matplotlib.pyplot as plt
import os
import cv2 
import scipy.io as sio
import scipy.ndimage
import glob

In [3]:
input_path = '/home/diya/Projects/super_resolution/flow_super_resolution/dataset/train_data_ml_spatio-temporal_fukami_paper/data_HR/'
output_path = '/home/diya/Projects/super_resolution/flow_super_resolution/dataset/train_data_ml_spatio-temporal_fukami_paper/outputs/bicubic/'
if not os.path.exists(output_path):
    os.makedirs(output_path)

In [4]:
# Set the random seed for reproducibility
tf.keras.utils.set_random_seed(42)

In [7]:
# Load high-resolution data generated from MATLAB code
hr_files = sorted(glob.glob('/home/diya/Projects/super_resolution/flow_super_resolution/dataset/train_data_ml_spatio-temporal_fukami_paper/data_HR/matlab_data/data*'))

high_res_data = []
for file in hr_files:
    mat_data = sio.loadmat(file)
    # Extract vorticity field 'omg' as mentioned in the paper
    high_res_data.append(mat_data['omg'])
    
high_res_data = np.array(high_res_data)  # Shape: [n_samples, 128, 128]
print(high_res_data.shape)

(256, 128, 128)


In [25]:
from scipy.ndimage import zoom

def average_downsample(data, target_size=(16, 16)):
    """Perform average downsampling on the input data"""
    n_samples = data.shape[0]
    low_res_data = np.zeros((n_samples, target_size[0], target_size[1]))
    
    for i in range(n_samples):
        # Reshape to perform average pooling
        h, w = data[i].shape
        pool_size = (h // target_size[0], w // target_size[1])
        reshaped = data[i].reshape(target_size[0], pool_size[0], 
                                  target_size[1], pool_size[1])
        low_res_data[i] = reshaped.mean(axis=(1, 3))
    
    return low_res_data

# Generate low-resolution data
# low_res_data_8x8 = average_downsample(high_res_data, target_size=(8, 8))
low_res_data_16x16 = average_downsample(high_res_data, target_size=(16,16))

In [26]:
from sklearn.model_selection import train_test_split

# First split: Train (80%) and Temp (20%)
X_train, X_temp, y_train, y_temp = train_test_split(
    low_res_data_16x16, high_res_data, test_size=0.2, random_state=42
)

# Second split: Validation (10%) and Test (10%)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=42
)

# Check the sizes
print(f"Train: {len(X_train)}, Validation: {len(X_val)}, Test: {len(X_test)}")


Train: 204, Validation: 26, Test: 26


#### Funciton for bicubic upsampling

In [27]:
def bicubic_upsample(data, target_size=(128, 128)):
    """Perform bicubic interpolation to upsample low-resolution data."""
    upsampled_data = np.zeros((data.shape[0], target_size[0], target_size[1]))
    
    for i in range(data.shape[0]):
        upsampled_data[i] = scipy.ndimage.zoom(data[i], 
                                               (target_size[0]/data.shape[1], target_size[1]/data.shape[2]), 
                                               order=3)  # Bicubic interpolation
    
    return upsampled_data

#### Apply bicubic upsampling to the LR data

In [39]:
# Upsample from 8x8 to 128x128 using bicubic interpolation
upsampled_train_16x16 = bicubic_upsample(X_train, target_size=(128, 128))
upsampled_val_16x16 = bicubic_upsample(X_val, target_size=(128, 128))
upsampled_test_16x16 = bicubic_upsample(X_test, target_size = (128, 128))

# Check shapes
print(f"Bicubic Upsampled Training Data: {upsampled_train_16x16.shape}")
print(f"Bicubic Upsampled Validation Data: {upsampled_val_16x16.shape}")
print(f"Bicubic Upsampled Test Data: {upsampled_test_16x16.shape}")


Bicubic Upsampled Training Data: (204, 128, 128)
Bicubic Upsampled Validation Data: (26, 128, 128)
Bicubic Upsampled Test Data: (26, 128, 128)


#### Save the upsampled data

In [40]:
# Define save paths
curr_dir = '/home/diya/Projects/super_resolution/flow_super_resolution/dataset/train_data_ml_spatio-temporal_fukami_paper/data_HR/'
bicubic_save_dir = curr_dir + 'bicubic_upsampled_16x16/'
os.makedirs(bicubic_save_dir, exist_ok=True)

# Save the data
np.save(bicubic_save_dir + 'train.npy', upsampled_train_16x16)
np.save(bicubic_save_dir + 'val.npy', upsampled_val_16x16)

print(f"Saved upsampled datasets in {bicubic_save_dir}")


Saved upsampled datasets in /home/diya/Projects/super_resolution/flow_super_resolution/dataset/train_data_ml_spatio-temporal_fukami_paper/data_HR/bicubic_upsampled_16x16/


#### Visualize the upsampled data

In [ ]:
def visualize_and_save_samples(low_res_samples, high_res_samples, upsampled_samples=None, num_samples=5, dataset_type="Training", save_dir="visualizations", save=True):
    """
    Visualizes and optionally saves low-res vs upsampled (if provided) vs high-res samples.

    Parameters:
        - low_res_samples: Array of low-resolution images.
        - high_res_samples: Array of high-resolution images.
        - upsampled_samples: (Optional) Array of upsampled images.
        - num_samples: Number of samples to visualize.
        - dataset_type: "Training" or "Validation".
        - save_dir: Directory to save the visualization images.
        - save: Whether to save the images (default: True).
    """
    os.makedirs(save_dir, exist_ok=True)  # Ensure save directory exists

    indices = np.random.choice(len(low_res_samples), num_samples, replace=False)

    for i, idx in enumerate(indices):
        num_cols = 3 if upsampled_samples is not None else 2  # Determine columns based on upsampling availability
        fig, axes = plt.subplots(1, num_cols, figsize=(15, 5))
        fig.suptitle(f"{dataset_type} Sample {i+1}", fontsize=14)

        # Low-res Image
        axes[0].imshow(low_res_samples[idx], cmap='inferno')
        axes[0].set_title(f"Low-Res {low_res_samples[idx].shape}")
        axes[0].axis('off')

        # Upsampled Image (if provided)
        if upsampled_samples is not None:
            axes[1].imshow(upsampled_samples[idx], cmap='inferno')
            axes[1].set_title(f"Upsampled {upsampled_samples[idx].shape}")
            axes[1].axis('off')

        # High-res Image
        axes[-1].imshow(high_res_samples[idx], cmap='inferno')
        axes[-1].set_title(f"High-Res {high_res_samples[idx].shape}")
        axes[-1].axis('off')

        # Save or show plot
        if save:
            save_path = os.path.join(save_dir, f"{dataset_type.lower()}_sample_{i+1}.png")
            plt.savefig(save_path, bbox_inches='tight')
            plt.close(fig)
        else:
            pass
            # plt.show()

        # print(f"Saved {dataset_type} Sample {i+1} to {save_dir}") if save else None

    print(f"Visualization completed for {num_samples} samples.")


In [ ]:
# Visualize bicubic upsampled results
save_dir = '/home/diya/Projects/super_resolution/flow_super_resolution/dataset/train_data_ml_spatio-temporal_fukami_paper/outputs/bicubic/visualizations_16x16/all/'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

Saved Test Sample 1 to /home/diya/Projects/super_resolution/flow_super_resolution/dataset/train_data_ml_spatio-temporal_fukami_paper/outputs/bicubic/visualizations_16x16/all/
Saved Test Sample 2 to /home/diya/Projects/super_resolution/flow_super_resolution/dataset/train_data_ml_spatio-temporal_fukami_paper/outputs/bicubic/visualizations_16x16/all/
Saved Test Sample 3 to /home/diya/Projects/super_resolution/flow_super_resolution/dataset/train_data_ml_spatio-temporal_fukami_paper/outputs/bicubic/visualizations_16x16/all/
Saved Test Sample 4 to /home/diya/Projects/super_resolution/flow_super_resolution/dataset/train_data_ml_spatio-temporal_fukami_paper/outputs/bicubic/visualizations_16x16/all/
Saved Test Sample 5 to /home/diya/Projects/super_resolution/flow_super_resolution/dataset/train_data_ml_spatio-temporal_fukami_paper/outputs/bicubic/visualizations_16x16/all/
Saved Test Sample 6 to /home/diya/Projects/super_resolution/flow_super_resolution/dataset/train_data_ml_spatio-temporal_fukam

In [ ]:
visualize_and_save_samples(X_train, y_train, upsampled_samples=upsampled_train_16x16, num_samples=len(X_train), dataset_type="Training", save_dir=save_dir)
visualize_and_save_samples(X_val, y_val, upsampled_samples=upsampled_val_16x16, num_samples=len(X_val), dataset_type="Validation", save_dir=save_dir)
visualize_and_save_samples(X_test, y_test, upsampled_samples=upsampled_test_16x16, num_samples=len(X_test), dataset_type="Test", save_dir=save_dir)